In [21]:
import torch
from torch import nn
from torch.autograd import Variable
import numpy as np
# from ret_benchmark.losses.registry import LOSS
# from ret_benchmark.utils.log_info import log_info


# @LOSS.register("contrastive_loss")
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()
        self.margin = 0.5

    def forward(self, inputs_col, targets_col):
        inputs_col = torch.cat(torch.unbind(inputs_col, dim=1), dim=0)
        targets_col = torch.cat([targets_col,targets_col])
        print(targets_col)
        inputs_row, target_row = inputs_col, targets_col
        n = inputs_col.size(0)
        # Compute similarity matrix
        sim_mat = torch.matmul(inputs_col, inputs_row.t())
        epsilon = 1e-5
        loss = list()

        neg_count = list()
        for i in range(n):
            pos_pair_ = torch.masked_select(sim_mat[i], targets_col[i] == target_row)
            pos_pair_ = torch.masked_select(pos_pair_, pos_pair_ < 1 - epsilon)
            neg_pair_ = torch.masked_select(sim_mat[i], targets_col[i] != target_row)

            neg_pair = torch.masked_select(neg_pair_, neg_pair_ > self.margin)

            pos_loss = torch.sum(-pos_pair_ + 1)
            if len(neg_pair) > 0:
                neg_loss = torch.sum(neg_pair)
                neg_count.append(len(neg_pair))
            else:
                neg_loss = 0

            loss.append(pos_loss + neg_loss)
        print(1)
        # if inputs_col.shape[0] == inputs_row.shape[0]:
        #     prefix = "batch_"
        # else:
        #     prefix = "memory_"
        # # if len(neg_count) != 0:
        #     log_info[f"{prefix}average_neg"] = sum(neg_count) / len(neg_count)
        # else:
        #     log_info[f"{prefix}average_neg"] = 0
        # log_info[f"{prefix}non_zero"] = len(neg_count)
        loss = sum(loss) / n  # / all_targets.shape[1]
        return loss

def gen_data(bs, nv, d, c) :
    xs = torch.rand(bs, nv, d)
    ys = torch.randint(0, c, (bs, ))      
    xs.requires_grad_(True)
    return xs, ys

bs, nv, d, c = 16, 2, 16, 10
logits, label = gen_data(bs, nv, d, c)
# logits = torch.squeeze(logits)
print(label)
print(logits.shape)
loss = Loss()(logits, label)
loss.backward()
print(loss)   

tensor([5, 3, 1, 4, 6, 4, 4, 4, 5, 3, 2, 4, 1, 2, 2, 5])
torch.Size([16, 2, 16])
tensor([5, 3, 1, 4, 6, 4, 4, 4, 5, 3, 2, 4, 1, 2, 2, 5, 5, 3, 1, 4, 6, 4, 4, 4,
        5, 3, 2, 4, 1, 2, 2, 5])
1
tensor(2.8503, grad_fn=<DivBackward0>)


tensor([4, 2, 1, 0, 3, 4, 0, 8, 0, 7, 8, 5, 4, 3, 8, 5])
torch.Size([16, 2, 16])
(tensor([[0.3824, 0.0655, 0.5452, 0.0421, 0.6513, 0.5132, 0.5871, 0.4720, 0.1851,
         0.4916, 0.2744, 0.1547, 0.6615, 0.8026, 0.1195, 0.6112],
        [0.4206, 0.8958, 0.6608, 0.9208, 0.1771, 0.2190, 0.0054, 0.1661, 0.2441,
         0.8426, 0.8671, 0.8522, 0.4704, 0.4017, 0.8272, 0.5905],
        [0.6965, 0.1619, 0.9814, 0.8449, 0.6499, 0.2081, 0.6850, 0.5897, 0.5989,
         0.2801, 0.3973, 0.5984, 0.9005, 0.1255, 0.9620, 0.7048],
        [0.0292, 0.8827, 0.7805, 0.9655, 0.5377, 0.6211, 0.8064, 0.9332, 0.6489,
         0.8588, 0.5309, 0.4430, 0.4684, 0.8366, 0.6080, 0.1456],
        [0.2274, 0.6891, 0.4976, 0.6588, 0.6310, 0.5824, 0.0983, 0.1417, 0.0042,
         0.3324, 0.8202, 0.6912, 0.2123, 0.3061, 0.0443, 0.7090],
        [0.1678, 0.6298, 0.1818, 0.6747, 0.4092, 0.0872, 0.8836, 0.1598, 0.7189,
         0.7128, 0.5129, 0.8580, 0.2035, 0.6982, 0.7599, 0.8857],
        [0.0356, 0.9228, 0.3994, 0.2

AttributeError: 'tuple' object has no attribute 'size'